## 153 Null Count

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error
from mv_wg_avg import exp_weight_avg


os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [2]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
df_hist = utils.read_df_pkl('../input/histori*0*')
df_new = utils.read_df_pkl('../input/new_mer*0*')

df_train = utils.reduce_mem_usage(df_train)
df_test  = utils.reduce_mem_usage(df_test )
df_hist  = utils.reduce_mem_usage(df_hist )
df_new   = utils.reduce_mem_usage(df_new  )

100%|██████████| 3/3 [00:00<00:00,  3.49it/s]


Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)
Mem. usage decreased to 1971.22 Mb (40.8% reduction)
Mem. usage decreased to 140.41 Mb (37.5% reduction)


In [3]:
df = df_new
df.sort_values(by=[key, 'purchase_date'], inplace=True)
df['authorized_flag'] =  df['authorized_flag'].map(lambda x: 0 if x=='N' else 1)
df_new = df.copy()
del df
gc.collect()

df = df_hist
df['authorized_flag'] =  df['authorized_flag'].map(lambda x: 0 if x=='N' else 1)
df['category_1'] =  df['category_1'].map(lambda x: 0 if x=='N' else 1)
df_hist = df.copy()
del df
gc.collect()
df_hist.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,1,C_ID_4e6213e9bc,88,0,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,1,C_ID_4e6213e9bc,88,0,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [4]:
# df_trans = pd.concat([df_hist, df_new], axis=0)
# df_trans = reduce_mem_usage(df_trans)
auth1 = df_hist[df_hist['authorized_flag']==1]
auth0 = df_hist[df_hist['authorized_flag']==0]

In [5]:
auth1[auth1.isnull().any(axis=1)].head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
11,1,C_ID_4e6213e9bc,-1,0,0,A,560,M_ID_7c5e93af2f,0,-0.727929,2018-02-08 20:05:45,NaN,-1,34
30,1,C_ID_4e6213e9bc,-1,0,0,A,80,M_ID_98b342c0e3,0,-0.724368,2018-02-27 09:04:48,NaN,-1,37
43,1,C_ID_4e6213e9bc,-1,0,0,A,278,NaN,0,-0.719154,2018-02-24 10:52:32,NaN,-1,37
80,1,C_ID_4e6213e9bc,-1,0,0,A,80,M_ID_98b342c0e3,0,-0.724368,2018-02-23 09:20:59,NaN,-1,37
82,1,C_ID_4e6213e9bc,-1,0,0,A,278,NaN,0,-0.713173,2018-02-25 12:07:37,NaN,-1,37


In [6]:
cnt_cols = ['city_id', 'state_id', 'subsector_id', 'installments']
cnt_cols = ['merchant_category_id']
for col in cnt_cols:
    auth1[col] =  auth1[col].replace(-1, np.nan)
    auth0[col] =  auth0[col].replace(-1, np.nan)
    df_new[col] =  df_new[col].replace(-1, np.nan)

### Null Count

In [7]:
for fname, df in zip(['auth1', 'auth0', 'new'], [auth1, auth0, df_new]):
    tmp = df[df.isnull().any(axis=1)]
    df_nan = pd.concat([tmp[key], (~(tmp==tmp)).sum(axis=1)], axis=1).rename(columns={0:f'{fname}_null_cnt'})
    df_nan = df_nan.groupby(key)[f'{fname}_null_cnt'].sum()
    df_train = df_train.join(df_nan)
    df_test = df_test.join(df_nan)
    
null_cols = [col for col in df_train.columns if col.count('null_')]
df_train[null_cols].fillna(0, inplace=True)
df_train['all_null_cnt'] = df_train[null_cols].sum(axis=1)
df_test['all_null_cnt'] = df_test[null_cols].sum(axis=1)
# df_train.drop(null_cols, axis=1, inplace=True)
df_train.fillna(0, inplace=True)
df_test.fillna(0, inplace=True)
df_train.head(10)

,first_active_month,feature_1,feature_2,feature_3,target,auth1_null_cnt,auth0_null_cnt,new_null_cnt,all_null_cnt
card_id,,,,,,,,,
C_ID_92a2005557,2017-06,5,2,1,-0.820312,5.0,1.0,0.0,6.0
C_ID_3d0044924f,2017-01,4,1,0,0.392822,31.0,2.0,0.0,33.0
C_ID_d639edf6cd,2016-08,2,2,0,0.687988,0.0,0.0,0.0,0.0
C_ID_186d6a6901,2017-09,4,3,0,0.142456,14.0,0.0,1.0,15.0
C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159790,13.0,3.0,1.0,17.0
C_ID_0894217f2f,2016-09,4,2,0,0.871582,1.0,0.0,0.0,1.0
C_ID_7e63323c00,2016-12,3,2,1,0.230103,1.0,0.0,0.0,1.0
C_ID_dfa21fc124,2017-09,3,2,1,2.136719,0.0,0.0,0.0,0.0
C_ID_fe0fdac8ea,2017-08,2,1,0,-0.065430,0.0,4.0,0.0,4.0


In [8]:
cnt_cols = ['category_2', 'category_3', 'merchant_category_id', 'city_id', 'state_id', 'subsector_id', 'installments']
for fname, df in zip(['auth1', 'auth0', 'new'],[auth1, auth0, df_new]):
    for col in cnt_cols:
        tmp = df[[key, col]]
        tmp = tmp[tmp.isnull().any(axis=1)]
        df_nan = pd.concat([tmp[key], (~(tmp==tmp)).sum(axis=1)], axis=1).rename(columns={0:f'{fname}_{col}_null_cnt'})
        df_nan = df_nan.groupby(key)[f'{fname}_{col}_null_cnt'].sum()
        df_train = df_train.join(df_nan)
        df_test = df_test.join(df_nan)
    
null_cols = [col for col in df_train.columns if col.count('null_')]
df_train[null_cols].fillna(0, inplace=True)
df_train['all_null_cnt'] = df_train[null_cols].sum(axis=1)
df_test['all_null_cnt'] = df_test[null_cols].sum(axis=1)
# df_train.drop(null_cols, axis=1, inplace=True)
df_train.fillna(0, inplace=True)
df_test.fillna(0, inplace=True)
df_train.head()

,first_active_month,feature_1,feature_2,feature_3,target,auth1_null_cnt,auth0_null_cnt,new_null_cnt,all_null_cnt,auth1_category_2_null_cnt,auth1_category_3_null_cnt,auth1_merchant_category_id_null_cnt,auth1_city_id_null_cnt,auth1_state_id_null_cnt,auth1_subsector_id_null_cnt,auth1_installments_null_cnt,auth0_category_2_null_cnt,auth0_category_3_null_cnt,auth0_merchant_category_id_null_cnt,auth0_city_id_null_cnt,auth0_state_id_null_cnt,auth0_subsector_id_null_cnt,auth0_installments_null_cnt,new_category_2_null_cnt,new_category_3_null_cnt,new_merchant_category_id_null_cnt,new_city_id_null_cnt,new_state_id_null_cnt,new_subsector_id_null_cnt,new_installments_null_cnt
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,2017-06,5,2,1,-0.820312,5.0,1.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C_ID_3d0044924f,2017-01,4,1,0,0.392822,31.0,2.0,0.0,99.0,29.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C_ID_d639edf6cd,2016-08,2,2,0,0.687988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C_ID_186d6a6901,2017-09,4,3,0,0.142456,14.0,0.0,1.0,45.0,12.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159790,13.0,3.0,1.0,50.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [10]:
df_train['category_2_city_diff_null_cnt'] = df_train['auth1_category_2_null_cnt'] - df_train['auth1_city_id_null_cnt']
df_test['category_2_city_diff_null_cnt'] = df_test['auth1_category_2_null_cnt'] - df_test['auth1_city_id_null_cnt']

null_cols = [col for col in df_train.columns if col.count('null_') and ( col.count('city') or col.count('category_3') or col.count('merchant') or col.count('install') )]
df_train['null_cnt'] = df_train[null_cols].sum(axis=1)
df_test['null_cnt'] = df_test[null_cols].sum(axis=1)

null_cols = [col for col in df_train.columns if col.count('auth1_') and ( col.count('city') or col.count('category_3') or col.count('merchant') or col.count('install') )]
df_train['auth1_null_cnt'] = df_train[null_cols].sum(axis=1)
df_test['auth1_null_cnt'] = df_test[null_cols].sum(axis=1)

null_cols = [col for col in df_train.columns if col.count('auth0_') and ( col.count('city') or col.count('category_3') or col.count('merchant') or col.count('install') )]
df_train['auth0_null_cnt'] = df_train[null_cols].sum(axis=1)
df_test['auth0_null_cnt'] = df_test[null_cols].sum(axis=1)

null_cols = [col for col in df_train.columns if col.count('new_') and ( col.count('city') or col.count('category_3') or col.count('merchant') or col.count('install') )]
df_train['new_null_cnt'] = df_train[null_cols].sum(axis=1)
df_test['new_null_cnt'] = df_test[null_cols].sum(axis=1)

SystemExit: 

### Save feature

In [14]:
fname = '153_nlc'
ignore_features = ['first_active_month', 'card_id', target]
df_feat = pd.concat([df_train, df_test], axis=0)

for col in df_feat.columns:
    if (col.count('state')):continue
    if (col.count('subsec')):continue
    if (col.count('all')):continue
    if (col.count('category_2_null')):continue
    if not(col.count('null_')):continue
    length = len(df_train[col].dropna())
    if length/len(df_train)<0.1:
        print(col)
        continue

    df_feat[col].fillna(0, inplace=True)
    feature = df_feat[col].values.astype('float32')
    if feature.shape[0]!=325540:
        print(feature.shape)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_all_{col}@', obj=feature)